---
teorias para como lidar com esta conspiração

tentar (ver como)
manter modelo 1 mas aumentar a % de (1 e) 5 pois são os q têm mais certeza
teoria, sendo q 5 têm mais concenço sobre ser mau
em vez de usar a média usar cada uma como sendo um nódulo diff

---



acrescentar mais nudulos aos já existentes

guardar numero de nodulos existentes  n = ...  
ou ver maior ID(de nódulo)

nodulo para todos os nodulos :

	se classificado com malignanci == 5:
		n = multiply(nodulo, n) 


multiply(nodulo, n):

	apinião para todas as opiniões:
		se opinião == malign
			criar nodulo n+1 => imagem e mascara da opinião
			acrescentá-la a nn_data\images a imagem e mascara
			n +=1
	return n
	
ver como obter radiomics e annotations_ds of seperate ones(worst case add copy of the mean)

In [2]:
import pylidc as pl
#from pylidc.utils import consensus
import matplotlib.pyplot as plt
import numpy as np
#import os
#from pathlib import Path
from utils import get_nodule_class, filename_from_annotation, cluster_annots

c:\Users\Daniela\Desktop\Fac\3ºano\Lab_IACD\Project1\lung-cancer-classification\utils.py:20: SyntaxWarning: invalid escape sequence '\_'
  """


In [17]:
anns = pl.query(pl.Annotation).filter(pl.Annotation.malignancy == 5)

'''
print(anns.count())
ann = anns.first()

print(ann.malignancy) 

for ann in anns:
    get_img(ann.scan.patient_id)
    

'''

print(anns)


SELECT annotations.id AS annotations_id, annotations.scan_id AS annotations_scan_id, annotations._nodule_id AS annotations__nodule_id, annotations.subtlety AS annotations_subtlety, annotations."internalStructure" AS "annotations_internalStructure", annotations.calcification AS annotations_calcification, annotations.sphericity AS annotations_sphericity, annotations.margin AS annotations_margin, annotations.lobulation AS annotations_lobulation, annotations.spiculation AS annotations_spiculation, annotations.texture AS annotations_texture, annotations.malignancy AS annotations_malignancy 
FROM annotations 
WHERE annotations.malignancy = ?


In [1]:
print(":)")

:)


In [13]:

from pylidc.Scan import ClusterError

highest_annotation_id = 6858
scans = pl.query(pl.Scan).all()

for scan in scans:

    try:
        # Get all nodules for the scan with increased tolerance
        nodules = scan.cluster_annotations(tol=10)  # Increase the tolerance if needed
    except ClusterError:
        print(f"ClusterError: Skipping scan {scan.patient_id} due to clustering issue.")
        continue  # Skip this scan if clustering fails

    
    # Loop over each nodule
    for nodule in nodules:
        # Get the malignancy ratings for all annotations for the nodule
        malignancies = [ann.malignancy for ann in nodule]
        
        # Check if all annotations for this nodule have a malignancy score of 5
        ''' 
        pk tem q ser todos? => faz média de várias opiniões e dá 5
        '''

        #makee sure that all docks think it is bad
        if malignancies.count(5) >= 3 and malignancies.count(4) <= 1: #all(m == 5 for m in malignancies)
            print(f"Nodule in scan {scan.patient_id} with malignancy 5:")
            print(f"Nodule details: {nodule}")

            #to be continued
            ''' 
            criar nodulo n+1 => imagem e mascara da opinião
			acrescentá-la a nn_data\images a imagem e mascara
			n +=1
	        return n

            ask to get the img of each nodule and the mask, give new id as (highest id +1)
            '''

            #untested lacks the save img and mask part
            # Now, create new nodules for each annotation that meets the criteria
            for ann in nodule:
                # Increment the nodule counter to assign unique IDs
                highest_annotation_id += 1
                
                # Here you would treat this annotation as a separate "nodule"
                print(f"New Nodule ID: {highest_annotation_id}")
                print(f"Annotation details: malignancy={ann.malignancy}, "
                      f"center=({ann.centroid()})")
                
                # You can store or process the annotation as a separate "nodule"
                # For example, save or analyze the annotation data here


    

Nodule in scan LIDC-IDRI-0007 with malignancy 5:
Nodule details: [Annotation(id=125,scan_id=18)]
Nodule in scan LIDC-IDRI-0020 with malignancy 5:
Nodule details: [Annotation(id=265,scan_id=31)]
Nodule in scan LIDC-IDRI-0029 with malignancy 5:
Nodule details: [Annotation(id=314,scan_id=40)]
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
ClusterError: Skipping scan LIDC-IDRI-0055 due to clustering issue.
Nodule in scan LIDC-IDRI-0058 with malignancy 5:
Nodule details: [Annotation(id=606,scan_id=69), Annotation(id=609,scan_id=69), Annotation(id=612,scan_id=69)]
Nodule in scan LIDC-IDRI-0060 with malignancy 5:
Nodule details: [Annotation(id=620,scan_id=71), Annotation(id=626,scan_id=71)]
Nodule in scan LIDC-IDRI-0060 with malignancy 5:
Nodule details: [Annotation(id=625,scan_id=71)]
Nodule in scan LIDC-IDRI-0072 with malignancy 5:
Nodule details: [Annotation(id=725,scan_id=82), Annotation(id=726,scan_id=82), Annotation(id=727,scan_i

KeyboardInterrupt: 